In [4]:
from __future__ import print_function, unicode_literals, absolute_import, division
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import os
import glob
from keras.models import Model, load_model
from keras.layers import Input
from keras.layers.core import Lambda
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import backend as K
from csbdeep.utils import _raise, Path, axes_check_and_normalize,axes_dict, move_image_axes, move_channel_for_backend, backend_channels_last

import tensorflow as tf
from tifffile import imread
from csbdeep.utils import Path, download_and_extract_zip_file, plot_some
from csbdeep.io import save_tiff_imagej_compatible
from csbdeep.models import CARE

In [5]:
def load_alltraining_data(file, axes=None, n_images=None, verbose=False):
    """Load training data from file in ``.npz`` format.

    The data file is expected to have the keys:

    - ``X``    : Array of training input images.
    - ``Y``    : Array of corresponding target images.
    - ``axes`` : Axes of the training images.


    Parameters
    ----------
    file : str
        File name

    axes: str, optional
        Must be provided in case the loaded data does not contain ``axes`` information.
    n_images : int, optional
        Can be used to limit the number of images loaded from data.
    verbose : bool, optional
        Can be used to display information about the loaded images.

    Returns
    -------
    tuple( tuple(:class:`numpy.ndarray`, :class:`numpy.ndarray`), tuple(:class:`numpy.ndarray`, :class:`numpy.ndarray`), str )
        Returns two tuples (`X_train`, `Y_train`), (`X_val`, `Y_val`) of training and validation sets
        and the axes of the input images.
        The tuple of validation data will be ``None`` if ``validation_split = 0``.

    """

    f = np.load(file)
    X, Y = f['X'], f['Y']
    if axes is None:
        axes = f['axes']
    axes = axes_check_and_normalize(axes)

    assert X.shape == Y.shape
    assert len(axes) == X.ndim
    assert 'C' in axes
    if n_images is None:
        n_images = X.shape[0]
    assert X.shape[0] == Y.shape[0]
    assert 0 < n_images <= X.shape[0]

    X, Y = X[:n_images], Y[:n_images]
    channel = axes_dict(axes)['C']

 

    X = move_channel_for_backend(X,channel=channel)
    Y = move_channel_for_backend(Y,channel=channel)

    axes = axes.replace('C','') # remove channel
    if backend_channels_last():
        axes = axes+'C'
    else:
        axes = axes[:1]+'C'+axes[1:]

    data_val =  None

    if verbose:
        ax = axes_dict(axes)
        n_train = len(X)
        image_size = tuple( X.shape[ax[a]] for a in 'TZYX' if a in axes )
        n_dim = len(image_size)
        n_channel_in, n_channel_out = X.shape[ax['C']], Y.shape[ax['C']]

        print('number of training images:\t', n_train)
     
        print('image size (%dD):\t\t'%n_dim, image_size)
        print('axes:\t\t\t\t', axes)
        print('channels in / out:\t\t', n_channel_in, '/', n_channel_out)

    return (X,Y),  axes


In [6]:
(X,Y), axes = load_alltraining_data('/data/u934/service_imagerie/v_kapoor/StarDistTraining/TestUnetSegmentation.npz', verbose=True)

c = axes_dict(axes)['C']
n_channel_in, n_channel_out = X.shape[c], Y.shape[c]
IMG_HEIGHT = X.shape[1]
IMG_WIDTH = X.shape[2]
IMG_CHANNELS = X.shape[c]
print(IMG_HEIGHT , IMG_WIDTH, IMG_CHANNELS )

number of training images:	 100
image size (2D):		 (64, 64)
axes:				 SYXC
channels in / out:		 1 / 1
64 64 1


In [7]:
basedir = '/data/u934/service_imagerie/v_kapoor/StarDistData/TestData/'
basedirResultsUnet = '/data/u934/service_imagerie/v_kapoor/StarDistData/JuliaResultsData/Unet/'



BaseDirModels = 'models/TestUNet_model.h5'

In [9]:
Path = os.path.join(basedir, '*.tif')
X = []
Names = []

filesRaw = glob.glob(Path)

for fname in filesRaw:
     x = imread(fname)
     X.append(x)
     Names.append(fname)
X.sort


axes = 'YX'
print('Total number of images to be segmented = ', len(X))
print('Image size = ', X[0].shape)
count = min(0, len(X))

Total number of images to be segmented =  3
Image size =  (512, 640)


In [10]:
def mean_iou(y_true, y_pred):
    prec = []
    for t in np.arange(0.5, 1.0, 0.05):
        y_pred_ = tf.to_int32(y_pred > t)
        score, up_opt = tf.metrics.mean_iou(y_true, y_pred_, 2)
        K.get_session().run(tf.local_variables_initializer())
        with tf.control_dependencies([up_opt]):
            score = tf.identity(score)
        prec.append(score)
    return K.mean(K.stack(prec), axis=0)

In [11]:
model = load_model(BaseDirModels, custom_objects={'mean_iou': mean_iou})


In [13]:
%%time
Segmentedfiles = []
sizes_test = []
for x in X:
    
    x = np.zeros((1,) + (x.shape[0], x.shape[1]) + (1,))
    segmented = model.predict(x, verbose=0)
    Segmentedfiles.append(segmented)

ValueError: Error when checking input: expected input_1 to have shape (64, 64, 1) but got array with shape (512, 640, 1)

In [27]:

for i in range(0, len(Names)):
   print('Saving file' +  basedirResultsUnet + '%s_' + os.path.basename(Names[i]))
   save_tiff_imagej_compatible((basedirResultsUnet + '%s_' + os.path.basename(Names[i])) % model.name, Segmentedfiles[i], axes)

In [28]:
for  i  in  range(0,  count):
      plt.figure(figsize=(16,10))
      plot_some(np.stack([Segmentedfiles[i]]),
          title_list=[['U-Net Segmentation']], 
          pmin=2,pmax=99.8);
      print('Filename = ', Names[i])